In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/data-storm-5-0/train_kaggle.csv
/kaggle/input/data-storm-5-0/test_kaggle.csv


In [2]:
# pip install scikit-learn

# Import libraries

In [3]:
from scipy import stats
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
customer_data = pd.read_csv("/kaggle/input/data-storm-5-0/train_kaggle.csv",low_memory=False)

In [5]:
customer_test_data = pd.read_csv("/kaggle/input/data-storm-5-0/test_kaggle.csv",low_memory=False)

In [6]:
customer_data.head()

,Customer_ID,outlet_city,luxury_sales,fresh_sales,dry_sales,cluster_catgeory
0,10493832.0,Kelaniya,1209.6,756,5292,4
1,10178643.0,Moratuwa,1590.12,1060.08,6007.12,1
2,10513916.0,Wattala,2288.88,1481.04,9155.52,4
3,10334589.0,Wattala,2274.94,1739.66,9099.76,4
4,10458365.0,Kelaniya,2345.49,2069.55,9243.99,4


In [7]:
customer_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 774155 entries, 0 to 774154
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Customer_ID       774153 non-null  float64
 1   outlet_city       774153 non-null  object 
 2   luxury_sales      774120 non-null  object 
 3   fresh_sales       774114 non-null  object 
 4   dry_sales         774125 non-null  object 
 5   cluster_catgeory  774154 non-null  object 
dtypes: float64(1), object(5)
memory usage: 35.4+ MB


In [8]:
missing_values = customer_data.isnull().sum()

# Print the count of missing values in each column
print("Missing values in each column:")
print(missing_values)

Missing values in each column:
Customer_ID          2
outlet_city          2
luxury_sales        35
fresh_sales         41
dry_sales           30
cluster_catgeory     1
dtype: int64


# Data Preprocessing

In [9]:

# Make a copy of the DataFrame to avoid warnings
customer_data_copy = customer_data.copy()

# Convert relevant columns to numeric
customer_data_copy['luxury_sales'] = pd.to_numeric(customer_data_copy['luxury_sales'], errors='coerce')
customer_data_copy['fresh_sales'] = pd.to_numeric(customer_data_copy['fresh_sales'], errors='coerce')
customer_data_copy['dry_sales'] = pd.to_numeric(customer_data_copy['dry_sales'], errors='coerce')
 

In [10]:
customer_data_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 774155 entries, 0 to 774154
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Customer_ID       774153 non-null  float64
 1   outlet_city       774153 non-null  object 
 2   luxury_sales      774110 non-null  float64
 3   fresh_sales       774105 non-null  float64
 4   dry_sales         774117 non-null  float64
 5   cluster_catgeory  774154 non-null  object 
dtypes: float64(4), object(2)
memory usage: 35.4+ MB


# Data Cleaning

In [11]:
# Display value counts of 'cluster_catgeory'
print(customer_data_copy['cluster_catgeory'].value_counts())

# Identify incorrect values (anything not in the range 1-6)
valid_values = ['1', '2', '3', '4', '5', '6']
customer_data_copy['cluster_catgeory'] = customer_data_copy['cluster_catgeory'].apply(lambda x: x if x in valid_values else np.nan)

# Forward fill NaN values with the previous row's value
customer_data_copy['cluster_catgeory'] =customer_data_copy['cluster_catgeory'].fillna(method='ffill')

# Check the value counts again to ensure corrections
print(customer_data_copy['cluster_catgeory'].value_counts())


cluster_catgeory
1      188984
4      172439
6      169214
2      155064
3       48907
5       39540
99          1
6\          1
95          1
98          1
100         1
89          1
Name: count, dtype: int64


/tmp/ipykernel_34/2171887483.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  customer_data_copy['cluster_catgeory'] =customer_data_copy['cluster_catgeory'].fillna(method='ffill')


cluster_catgeory
1    188986
4    172441
6    169215
2    155066
3     48907
5     39540
Name: count, dtype: int64


In [12]:

numeric_cols = ['luxury_sales', 'fresh_sales', 'dry_sales'] 

# Impute missing values with the mean for numerical columns
imputer = SimpleImputer(strategy='mean')
customer_data_copy[numeric_cols] = imputer.fit_transform(customer_data_copy[numeric_cols])

# Drop rows with missing values in 'Customer_ID' and 'outlet_city' columns
customer_data_copy.dropna(subset=['Customer_ID', 'outlet_city','cluster_catgeory'], inplace=True)

# Detect and handle outliers using Z-score
z_scores = np.abs(stats.zscore(customer_data_copy[numeric_cols]))
outliers = (z_scores > 3).any(axis=1)

# Print number of outliers
print(f'Number of outliers detected: {outliers.sum()}')

# Option 1: Remove outliers
customer_data_copy = customer_data_copy[~outliers]

# Option 2: Cap outliers (example with 99th percentile capping)
def cap_outliers(series):
    upper_limit = series.quantile(0.99)
    series = np.where(series > upper_limit, upper_limit, series)
    return series

customer_data_copy[numeric_cols] = customer_data_copy[numeric_cols].apply(lambda x: cap_outliers(x) if x.dtype in ['float64', 'int64'] else x)

# Check for inconsistencies and duplicates
# Remove duplicates if any
customer_data_copy.drop_duplicates(inplace=True)

Number of outliers detected: 16498


In [13]:
customer_data_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 757653 entries, 0 to 774150
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Customer_ID       757653 non-null  float64
 1   outlet_city       757653 non-null  object 
 2   luxury_sales      757653 non-null  float64
 3   fresh_sales       757653 non-null  float64
 4   dry_sales         757653 non-null  float64
 5   cluster_catgeory  757653 non-null  object 
dtypes: float64(4), object(2)
memory usage: 40.5+ MB


In [14]:
customer_data_copy['outlet_city'].value_counts()

outlet_city
Colombo                   39461
Batticaloa                39422
Galle                     39415
Kalmunai                  39189
Negombo                   39113
Jaffna                    39103
Katunayake                39077
Kandy                     38976
Trincomalee               38927
Gampaha                   38747
Peliyagoda                37065
Nuwara Eliya              37011
Dehiwala-Mount Lavinia    36768
Kotte                     36716
Panadura                  36644
Wattala                   36642
Kaduwela                  36604
Homagama                  36546
Moratuwa                  36338
Kelaniya                  35889
Name: count, dtype: int64

# Feature Engineering

In [15]:
# Feature Engineering: Creating new features
customer_data_copy['total_sales'] = customer_data_copy[numeric_cols].sum(axis=1)
customer_data_copy['average_sales'] = customer_data_copy[numeric_cols].mean(axis=1)
customer_data_copy['luxury_sales_ratio'] = customer_data_copy['luxury_sales'] / customer_data_copy['total_sales']
customer_data_copy['fresh_sales_ratio'] = customer_data_copy['fresh_sales'] / customer_data_copy['total_sales']
customer_data_copy['dry_sales_ratio'] = customer_data_copy['dry_sales'] / customer_data_copy['total_sales']
customer_data_copy['sales_variance'] = customer_data_copy[numeric_cols].var(axis=1)

# # Define lists for urban, suburban, and rural cities
# urban_cities = ["Colombo", "Dehiwala-Mount Lavinia", "Kotte", "Moratuwa", "Galle", "Katunayake", "Jaffna", "Kandy"]
# suburban_cities = ["Negombo", "Homagama", "Trincomalee", "Wattala", "Gampaha", "Kalmunai", "Batticaloa", "Panadura", "Nuwara Eliya"]
# rural_cities = ["Kaduwela", "Kelaniya", "Peliyagoda"]

# Define lists for urban, suburban, and rural cities
urban_cities = ["Colombo", "Kandy", "Galle", "Negombo", "Jaffna"]
suburban_cities = ["Kotte", "Dehiwala-Mount Lavinia", "Maharagama", "Panadura", "Kurunegala"]
rural_cities = ["Batticaloa", "Kalmunai", "Trincomalee", "Nuwara Eliya", "Moratuwa"]

customer_data_copy['city_group'] = customer_data_copy['outlet_city'].apply(lambda x: 2 if x in urban_cities else (1 if x in suburban_cities else 0))

# # Perform feature crossing by multiplying city_group with average_sales
# customer_data_copy['city_group_average_sales'] = customer_data_copy['city_group'] * customer_data_copy['average_sales']
# customer_data_copy['city_group_total_sales'] = customer_data_copy['city_group'] * customer_data_copy['total_sales']



# Normalization/Standardization: Scaling the data

In [16]:
scaler = StandardScaler()
# Select the columns to normalize/standardize
columns_to_normalize = ['luxury_sales_ratio', 'fresh_sales_ratio', 'dry_sales_ratio']
# Fit the scaler on the selected columns and transform the data
customer_data_copy[[ 'luxury_sales_ratio_scaled', 'fresh_sales_ratio_scaled', 'dry_sales_ratio_scaled']] = scaler.fit_transform(customer_data_copy[columns_to_normalize])

In [17]:
customer_data_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 757653 entries, 0 to 774150
Data columns (total 16 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Customer_ID                757653 non-null  float64
 1   outlet_city                757653 non-null  object 
 2   luxury_sales               757653 non-null  float64
 3   fresh_sales                757653 non-null  float64
 4   dry_sales                  757653 non-null  float64
 5   cluster_catgeory           757653 non-null  object 
 6   total_sales                757653 non-null  float64
 7   average_sales              757653 non-null  float64
 8   luxury_sales_ratio         757653 non-null  float64
 9   fresh_sales_ratio          757653 non-null  float64
 10  dry_sales_ratio            757653 non-null  float64
 11  sales_variance             757653 non-null  float64
 12  city_group                 757653 non-null  int64  
 13  luxury_sales_ratio_scaled  757653 

In [18]:
# # Splitting the data into features and target variable
# X = customer_data_copy[['luxury_sales_ratio', 'fresh_sales_ratio', 'dry_sales_ratio', 'city_group', 'city_group_average_sales_scaled', 'city_group_total_sales_scaled']]
# y = customer_data_copy[['cluster_catgeory']]

In [19]:
X = customer_data_copy.drop(['Customer_ID', 'outlet_city', 'cluster_catgeory'], axis=1)
y = customer_data_copy[['cluster_catgeory']]

In [20]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 757653 entries, 0 to 774150
Data columns (total 13 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   luxury_sales               757653 non-null  float64
 1   fresh_sales                757653 non-null  float64
 2   dry_sales                  757653 non-null  float64
 3   total_sales                757653 non-null  float64
 4   average_sales              757653 non-null  float64
 5   luxury_sales_ratio         757653 non-null  float64
 6   fresh_sales_ratio          757653 non-null  float64
 7   dry_sales_ratio            757653 non-null  float64
 8   sales_variance             757653 non-null  float64
 9   city_group                 757653 non-null  int64  
 10  luxury_sales_ratio_scaled  757653 non-null  float64
 11  fresh_sales_ratio_scaled   757653 non-null  float64
 12  dry_sales_ratio_scaled     757653 non-null  float64
dtypes: float64(12), int64(1)
memory us

In [21]:


# Split the data into training and validation sets (80% training, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Check the shapes of the resulting datasets
print("Shape of X_train:", X_train.shape)
print("Shape of X_val:", X_val.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_val:", y_val.shape)


# Initialize LabelEncoder
label_encoder = LabelEncoder()

Shape of X_train: (606122, 13)
Shape of X_val: (151531, 13)
Shape of y_train: (606122, 1)
Shape of y_val: (151531, 1)


In [22]:
# Fit LabelEncoder on the target variable and transform labels
y_train_encoded = label_encoder.fit_transform(y_train.values.ravel())
y_val_encoded = label_encoder.transform(y_val.values.ravel())

# Initialize XGBoost classifier
xgb_classifier = XGBClassifier()

# Train the classifier on the training data
xgb_classifier.fit(X_train, y_train_encoded)

# Predict the labels for the validation set
y_pred_encoded = xgb_classifier.predict(X_val)

# Calculate accuracy on the validation set
accuracy_encoded = accuracy_score(y_val_encoded, y_pred_encoded)
print("Accuracy on validation set:", accuracy_encoded)

Accuracy on validation set: 0.9767110360256317


# Test data Preprocessing 

In [23]:
customer_test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40749 entries, 0 to 40748
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Customer_ID   40749 non-null  int64 
 1   outlet_city   40749 non-null  object
 2   luxury_sales  40749 non-null  object
 3   fresh_sales   40749 non-null  object
 4   dry_sales     40749 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.6+ MB


In [24]:
customer_test_data["luxury_sales"].value_counts()

luxury_sales
1801.8     7
1560.6     6
1684.2     6
999.6      6
1492.64    6
          ..
1779.84    1
2256.96    1
1702.5     1
2611.54    1
4212.16    1
Name: count, Length: 33183, dtype: int64

In [25]:
numeric_cols = ['luxury_sales', 'fresh_sales', 'dry_sales'] 

In [27]:

# Step 1: Identify non-numeric entries
for col in ['luxury_sales', 'fresh_sales', 'dry_sales']:
    non_numeric = customer_test_data[pd.to_numeric(customer_test_data[col], errors='coerce').isna()]
    print(f"Non-numeric entries in {col} column:")
    print(non_numeric)

# Step 2: Replace non-numeric entries with NaN
for col in ['luxury_sales', 'fresh_sales', 'dry_sales']:
    customer_test_data[col] = pd.to_numeric(customer_test_data[col], errors='coerce')

# Print DataFrame after replacing non-numeric entries with NaN
print("\nDataFrame after replacing non-numeric entries with NaN:")
print(customer_test_data)

# Step 3: Impute missing values with the mean for numerical columns
numeric_cols = ['luxury_sales', 'fresh_sales', 'dry_sales']
imputer = SimpleImputer(strategy='mean')
customer_test_data[numeric_cols] = imputer.fit_transform(customer_test_data[numeric_cols])

# Print DataFrame after imputation
print("\nDataFrame after imputing missing values with mean:")
print(customer_test_data)

Non-numeric entries in luxury_sales column:
      Customer_ID   outlet_city                       luxury_sales  \
6           10364    batticaloa   Seven Hundread and sixty Ruppees   
6299         4059  Anuradhapura  Eight Hundread and fifteen ruppes   

     fresh_sales dry_sales  
6          914.4    3352.8  
6299     1548.69    5705.7  
Non-numeric entries in fresh_sales column:
       Customer_ID outlet_city luxury_sales            fresh_sales dry_sales
13949        28662      Jaffna      2195.84  Nine thousand ruppess   1509.64
Non-numeric entries in dry_sales column:
       Customer_ID outlet_city luxury_sales fresh_sales  \
40746        28664     Wattala       3838.6      3951.5   

                                   dry_sales  
40746  Three Thousand seven hundread ruppees  

DataFrame after replacing non-numeric entries with NaN:
       Customer_ID outlet_city  luxury_sales  fresh_sales  dry_sales
0            33574  batticaloa       2686.50      3582.00   12537.00
1           

In [29]:
customer_test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40749 entries, 0 to 40748
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Customer_ID   40749 non-null  int64  
 1   outlet_city   40749 non-null  object 
 2   luxury_sales  40749 non-null  float64
 3   fresh_sales   40749 non-null  float64
 4   dry_sales     40749 non-null  float64
dtypes: float64(3), int64(1), object(1)
memory usage: 1.6+ MB


In [30]:
# Feature Engineering: Creating new features
customer_test_data['total_sales'] = customer_test_data[numeric_cols].sum(axis=1)
customer_test_data['average_sales'] = customer_test_data[numeric_cols].mean(axis=1)
customer_test_data['luxury_sales_ratio'] = customer_test_data['luxury_sales'] / customer_test_data['total_sales']
customer_test_data['fresh_sales_ratio'] = customer_test_data['fresh_sales'] / customer_test_data['total_sales']
customer_test_data['dry_sales_ratio'] = customer_test_data['dry_sales'] / customer_test_data['total_sales']
customer_test_data['sales_variance'] = customer_test_data[numeric_cols].var(axis=1)

# # Define lists for urban, suburban, and rural cities
urban_cities = ["Colombo", "Kandy", "Galle", "Negombo", "Jaffna"]
suburban_cities = ["Kotte", "Dehiwala-Mount Lavinia", "Maharagama", "Panadura", "Kurunegala"]
rural_cities = ["Batticaloa", "Kalmunai", "Trincomalee", "Nuwara Eliya", "Moratuwa"]

customer_data_copy['city_group'] = customer_data_copy['outlet_city'].apply(lambda x: 2 if x in urban_cities else (1 if x in suburban_cities else 0))


# Create a new column 'city_group' and assign labels
customer_test_data['city_group'] = customer_test_data['outlet_city'].apply(lambda x: 2 if x in urban_cities else (1 if x in suburban_cities else 0))


In [31]:
# Normalization/Standardization: Scaling the data
scaler = StandardScaler()
# Select the columns to normalize/standardize
columns_to_normalize = ['luxury_sales_ratio', 'fresh_sales_ratio', 'dry_sales_ratio', ]
# Fit the scaler on the selected columns and transform the data
customer_test_data[[ 'luxury_sales_ratio_scaled', 'fresh_sales_ratio_scaled', 'dry_sales_ratio_scaled']] = scaler.fit_transform(customer_test_data[columns_to_normalize])

In [32]:

# X_test = customer_test_data[['luxury_sales_ratio', 'fresh_sales_ratio', 'dry_sales_ratio', 'city_group']]
X = customer_test_data.drop(['Customer_ID', 'outlet_city'], axis=1)

In [33]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40749 entries, 0 to 40748
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   luxury_sales               40749 non-null  float64
 1   fresh_sales                40749 non-null  float64
 2   dry_sales                  40749 non-null  float64
 3   total_sales                40749 non-null  float64
 4   average_sales              40749 non-null  float64
 5   luxury_sales_ratio         40749 non-null  float64
 6   fresh_sales_ratio          40749 non-null  float64
 7   dry_sales_ratio            40749 non-null  float64
 8   sales_variance             40749 non-null  float64
 9   city_group                 40749 non-null  int64  
 10  luxury_sales_ratio_scaled  40749 non-null  float64
 11  fresh_sales_ratio_scaled   40749 non-null  float64
 12  dry_sales_ratio_scaled     40749 non-null  float64
dtypes: float64(12), int64(1)
memory usage: 4.0 MB


In [34]:

# Predict the labels for the test set
y_test_pred = xgb_classifier.predict(X)

# Optionally, convert the encoded labels back to the original categories
y_test_pred_labels = label_encoder.inverse_transform(y_test_pred)


In [35]:
# Save only Customer_ID and predicted_cluster_catgeory to a CSV file
output_df = customer_test_data[['Customer_ID']].copy()
output_df['cluster category'] = y_test_pred_labels
output_df.to_csv('test_predictions.csv', index=False)

print("Predictions for the test set have been saved to 'test_predictions.csv'")

Predictions for the test set have been saved to 'test_predictions.csv'
